<a href="https://colab.research.google.com/github/Sasa-fras/odw/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab 2: Gravitational Waves

####Import Stuff

In [1]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
#Tutorial 1.1
! pip install -q 'gwosc==0.7.1'
import gwosc
from gwosc import datasets
from gwosc.datasets import find_datasets
from gwosc.datasets import event_gps
from gwosc.datasets import event_at_gps
from gwosc.datasets import run_segment
from gwosc.datasets import run_at_gps
from gwosc.locate import get_event_urls
from gwosc.datasets import query_events
#Tutorial 1.2 & 1.3
! pip install -q 'gwpy==3.0.12'
import gwpy
from gwpy.timeseries import TimeSeries
from scipy.signal import get_window
#Tutorial 1.4
! pip install -q 'lalsuite==7.25' 'PyCBC==2.4.1'
import matplotlib.pyplot as plt
from pycbc.waveform import get_td_waveform, get_fd_waveform, fd_approximants, td_approximants
#Tutorial 2.1
import numpy
# https://pycbc.org/pycbc/latest/html/noise.html
import pycbc.noise
import pycbc.psd
from pycbc.types import TimeSeries
#Tutorial 2.2
! pip install -q -U 'scipy==1.12.0'
from pycbc.catalog import Merger
from pycbc.filter import resample_to_delta_t, highpass
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.filter import matched_filter
from pycbc.filter import sigma
from pycbc.frame import read_frame
import os
import urllib
#Tutorial 2.3
# WARNING!! If you are having problems with this code, replace the import with
#from pycbc_chisq import power_chisq
from pycbc.vetoes import power_chisq
from pycbc.events.ranking import newsnr
from pycbc.detector import Detector
#Tutorial 3.1
! pip install -U -q 'bilby==2.4.0' 'matplotlib==3.10.0' 'dynesty==2.1.5' 'corner==2.2.3'
import numpy as np
# The first import of matplotlib can take some time (especially on cloud platforms). This is normal.
import matplotlib
import scipy
import bilby
import corner
#Tutorial 3.2
from bilby.core.prior import Uniform, PowerLaw
from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters, generate_all_bbh_parameters
#Tutorial 3.3
! pip install -U -q 'astropy==7.0.1'
import h5py
import pandas as pd
import astropy.units as u
from astropy.cosmology import Planck15, z_at_value
#Tutorial 3.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 56.0 MB/s eta 0:00:00


In [18]:
#Gather every single event and add it to a database where object oriented programing is used to attribute the correct catalog, run, and observatory to each event

#Create the empty dataframe
DATA = pd.DataFrame({'Catalog':[],'Run':[],'Detector':[],'Event':[],'Time':[],'gps':[]})


#Populate the dataframe
for r in find_datasets(type='run'):
  for c in find_datasets(type='catalog', segment=run_segment(r)):
    for e in find_datasets(type='event', catalog=c, segment=run_segment(r)):
      DATA.loc[len(DATA)] = [c, r, datasets.event_detectors(e), e, gwpy.time.tconvert(event_gps(e)), event_gps(e)]




In [17]:
#reduce the dataset by making sure only the highest version of each event is present, e.g. if there are 3 events with the same string except that one ends in -v1, -v2, -v3, then only the one that ends in -v3 should remain and then the name should be changed so the -vx is removed from the string

# Sort by Event in descending order to easily get the latest version
DATA = DATA.sort_values(by=['Event'], ascending=False)

# Drop duplicates based on the base event name (remove the -vx suffix for comparison)
# We keep the first occurrence after sorting, which is the latest version
DATA['Event_Base'] = DATA['Event'].str.replace(r'-v\d+$', '', regex=True)
DATA = DATA.drop_duplicates(subset=['Event_Base'], keep='first')

# Remove the version suffix from the Event column
DATA['Event'] = DATA['Event_Base']

# Drop the temporary 'Event_Base' column
DATA = DATA.drop(columns=['Event_Base'])

# Reset the index after dropping rows
DATA = DATA.reset_index(drop=True)

# Print the updated DataFrame head to show the result
display(DATA.head())

,Catalog,Run,Detector,Event,Time,gps,Object_Name
0,Initial_LIGO_Virgo,S6,"{H1, L1}",blind_injection,2010-09-16 06:42:22.900,9.686546e+08,_blind_injection
1,O4_Discovery_Papers,O4b3Disc_16KHZ_R1,"{H1, L1}",GW250114_082203,2025-01-14 08:22:03.200,1.420878e+09,_GW250114_082203
2,GWTC-4.0,O4a_4KHZ_R1,{H1},GW240109_050431,2024-01-09 05:04:31.800,1.388812e+09,_GW240109_050431
3,GWTC-4.0,O4a_4KHZ_R1,"{H1, L1}",GW240107_013215,2024-01-07 01:32:15.700,1.388626e+09,_GW240107_013215
4,GWTC-4.0,O4a_16KHZ_R1,{H1},GW240105_151143,2024-01-05 15:11:43.200,1.388503e+09,_GW240105_151143


In [11]:
#sort the datafram by time, and then by event if the times are identical
DATA = DATA.sort_values(by=['Time', 'Event'])
DATA['Catalog'] = DATA['Catalog'].astype(str)
DATA['Run'] = DATA['Run'].astype(str)
DATA['Event'] = DATA['Event'].astype(str)
DATA['gps'] = DATA['gps'].astype(float)

#reindex
DATA = DATA.reset_index(drop=True)

# Set options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000) # Adjust as needed for your console width

#print(DATA)

# Reset options to default if needed
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')

In [12]:
#Try using object oriented programming to create a Class called 'event' which can be used to more easily access the other variables associated with each event

class Event:
  def __init__(self, Catalog, Run, Detector, Event, Time, gps):
    self.Catalog = Catalog
    self.Run = Run
    self.Detector = Detector
    self.Event = Event
    self.Time = Time
    self.gps = gps

#if event name has a '-' replace it with a '_' for the event_name
#if event name has a '.' replace it with a '_' for the event_name
#if event name starts with a number, append a '_' in the front of the event_name
#actually, just put a '_' infront of every name to be consistent
for i in range(len(DATA)):
  event_name = DATA['Event'][i]
  if '-' in event_name:
    event_name = event_name.replace('-', '_')
  if '.' in event_name:
    event_name = event_name.replace('.', '_')
  event_name = '_' + event_name
  globals()[event_name] = Event(DATA['Catalog'][i], DATA['Run'][i], DATA['Detector'][i], DATA['Event'][i], DATA['Time'][i], DATA['gps'][i])

In [13]:
print(datasets.find_datasets(type='events'))

['151008-v1', '151012.2-v1', '151116-v1', '161202-v1', '161217-v1', '170208-v1', '170219-v1', '170405-v1', '170412-v1', '170423-v1', '170616-v1', '170630-v1', '170705-v1', '170720-v1', '190924_232654-v1', '191118_212859-v1', '191223_014159-v1', '191225_215715-v1', '200114_020818-v1', '200121_031748-v1', '200201_203549-v1', '200214_224526-v1', '200214_224526-v2', '200219_201407-v1', '200311_103121-v1', 'GRB051103-v1', 'GW150914-v1', 'GW150914-v2', 'GW150914-v3', 'GW151012-v1', 'GW151012-v2', 'GW151012-v3', 'GW151226-v1', 'GW151226-v2', 'GW170104-v1', 'GW170104-v2', 'GW170608-v1', 'GW170608-v2', 'GW170608-v3', 'GW170729-v1', 'GW170809-v1', 'GW170814-v1', 'GW170814-v2', 'GW170814-v3', 'GW170817-v1', 'GW170817-v2', 'GW170817-v3', 'GW170818-v1', 'GW170823-v1', 'GW190403_051519-v1', 'GW190408_181802-v1', 'GW190408_181802-v2', 'GW190408_181802-v3', 'GW190412-v1', 'GW190412-v2', 'GW190412-v3', 'GW190412-v5', 'GW190412_053044-v4', 'GW190413_052954-v1', 'GW190413_052954-v2', 'GW190413_052954-v3'

Note that technically there are actually different versions of many of the events. Each version of each event has the same information (Catalog, Run, Detectors, Time, gps). The data you pull from the event using gwpy or gwosc may be different, but you should only be working with the highest version of each event, so the dataset was truncated to only include the highest versions, and to remove the annoying -vx tag at the end of the string. If you don't include the version tag when using gwpy or gwosc, it still works and automatically uses the highest version. Specifiation of version in the event string is only needed to access older versions.

In [14]:
print(type(_151012_2_v1.Detector))

<class 'set'>


When working with individual events such as "GW150914" it is easiest to get related information on that event using the object instance of it. The printed list below is the name of the object variable for each event. The naming scheme replaces any "." or "-" with _ and also puts a _ infront of every name since some start with numbers and variables can't. So for "GW150914" the object variable should be called "_GW150914". For the wierd case "151012.2" which has a period for some reason, the object variable name would be "_151012_2".

In [15]:
DATA['Object_Name'] = '_' + DATA['Event']
for i in range(len(DATA)):
  DATA.loc[i, 'Object_Name'] = DATA['Object_Name'][i].replace('-', '_')
  DATA.loc[i, 'Object_Name'] = DATA['Object_Name'][i].replace('.', '_')
print(DATA['Object_Name'])

0      _blind_injection
1             _GW150914
2               _151008
3             _151012_2
4             _GW151012
             ...       
261    _GW240104_164932
262    _GW240105_151143
263    _GW240107_013215
264    _GW240109_050431
265    _GW250114_082203
Name: Object_Name, Length: 266, dtype: object


How to access information about an event inside of a loop, such that analysis can be done with many events with minimal code.

In [16]:

for i in range(len(DATA)):
  print(globals()[DATA['Object_Name'][i]].Event)
  #globals() is the dictionary
  #globals()['name'] is the variable in the dictionary with name 'name'
  #globals()['name'].Event is the Event attribute of the variable (object) 'name' in the case that the variable is an instance of the class Event (so an object)

blind_injection
GW150914
151008
151012.2
GW151012
151116
GW151226
161202
161217
GW170104
170208
170219
170405
170412
170423
GW170608
170616
170630
170705
170720
GW170729
GW170809
GW170814
GW170817
GW170818
GW170823
GW190403_051519
GW190408_181802
GW190412
GW190412_053044
GW190413_052954
GW190413_134308
GW190421_213856
GW190424_180648
GW190425
GW190425_081805
GW190426_152155
GW190426_190642
GW190503_185404
GW190512_180714
GW190513_205428
GW190514_065416
GW190517_055101
GW190519_153544
GW190521
GW190521_030229
GW190521_074359
GW190527_092055
GW190531_023648
GW190602_175927
GW190620_030421
GW190630_185205
GW190701_203306
GW190704_104834
GW190706_222641
GW190707_083226
GW190707_093326
GW190708_232457
GW190711_030756
GW190718_160159
GW190719_215514
GW190720_000836
GW190725_174728
GW190727_060333
GW190728_064510
GW190731_140936
GW190803_022701
GW190805_211137
GW190814_192009
GW190814
GW190814_211039
GW190818_232544
GW190821_124821
GW190828_063405
GW190828_065509
GW190906_054335
GW190909_1141

In this case the code prints the actual Event string of each event, the string that needs to be used in gwosc or gwpy functions (as apposed to the event Object_Name which has been typeset to be a valid variable name). To instead print other information associated with the event, ".Event" could be replaced with ".Catalog", ".Run", ".Detector", ".Time", or ".gps". Each of which will return the associated string or value that can be used by gwosc or gwpy.

#GW150914 Analysis (Bare Minimum)


###Choose cuts and lters that clean up the data as best you can.
Be methodical and take your time with this, trying many different ways to clean your data. This will be the most important step to improve your uncertainties!

###Create a simple oscillating function and use it to fit the data.
This can be done in multiple steps if you separate the major sections of the data in a clever manner, or all at once depending on the complexity of your function.

###Determine the chirp mass of the merger events, making sure to correctly determine the uncertainty of the chirp mass from the experimental data.

Make sure to include the detectors strain noise as part of your error analysis, and how your cuts affect the signal-to-noise ratio.

Determine the mass rest by using the fit you made above, then by using the tools available in Bilby and compare the results from the two methods.

Plot and fit a Gaussian to your chirp mass to help determine your uncertainty.

#Other Shit

Gyatt Rizzler type shit fr fr

In [9]:
print(event_gps('GW190425-v1'))
print(event_gps('GW190425-v2'))

1240215503.0
1240215503.0
